<a href="https://colab.research.google.com/github/cstrachan342/Notebook-Library-/blob/notebooks/formal_text_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d dariocioni/c4200m

100% 14.5G/14.5G [08:58<00:00, 27.0MB/s]
100% 14.5G/14.5G [08:58<00:00, 28.9MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.flush_and_unmount()

In [ ]:
!mkdir /content/drive/MyDrive/formal_text

#CREATE DIR IN DRIVE

In [ ]:
!unzip /content/c4200m.zip -d /content/drive/MyDrive/formal_text

In [ ]:
!unzip /content/SentenceCorrection-MTGRU-Seq2Seq/datasets/Data-QnA/Dataset.zip -d /content/unzip_test

Archive:  /content/SentenceCorrection-MTGRU-Seq2Seq/datasets/Data-QnA/Dataset.zip
   creating: /content/unzip_test/Dataset/
   creating: /content/unzip_test/Dataset/kr_clean/
  inflating: /content/unzip_test/Dataset/kr_clean/qa20_agents-motivations_train.txt  
  inflating: /content/unzip_test/Dataset/kr_clean/qa20_agents-motivations_test.txt  
  inflating: /content/unzip_test/Dataset/kr_clean/qa19_path-finding_train.txt  
  inflating: /content/unzip_test/Dataset/kr_clean/qa19_path-finding_test.txt  
  inflating: /content/unzip_test/Dataset/kr_clean/qa18_size-reasoning_train.txt  
  inflating: /content/unzip_test/Dataset/kr_clean/qa18_size-reasoning_test.txt  
  inflating: /content/unzip_test/Dataset/kr_clean/qa17_positional-reasoning_train.txt  
  inflating: /content/unzip_test/Dataset/kr_clean/qa17_positional-reasoning_test.txt  
  inflating: /content/unzip_test/Dataset/kr_clean/qa16_basic-induction_train.txt  
  inflating: /content/unzip_test/Dataset/kr_clean/qa16_basic-induction_tes

In [ ]:
!git clone https://github.com/gcunhase/SentenceCorrection-MTGRU-Seq2Seq.git

Cloning into 'SentenceCorrection-MTGRU-Seq2Seq'...
remote: Enumerating objects: 789, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 789 (delta 2), reused 2 (delta 2), pack-reused 786
Receiving objects: 100% (789/789), 3.91 MiB | 23.03 MiB/s, done.
Resolving deltas: 100% (452/452), done.


In [ ]:
import pandas as pd
import string
from string import digits
import numpy as np
import plotly.express as px
from timeit import default_timer as timer
import os

import tensorflow as tf

from sklearn.model_selection import train_test_split

In [ ]:
incorrect = pd.read_csv('/content/SentenceCorrection-MTGRU-Seq2Seq/datasets/NUS Text Norm Corpus/wrong_english.txt', sep='delimiter', header=None, names=['original_text'], engine='python')

In [ ]:
correct = pd.read_csv('/content/SentenceCorrection-MTGRU-Seq2Seq/datasets/NUS Text Norm Corpus/correct_english.txt', sep='delimiter', header=None, names=['corrected_text'], engine='python')

In [ ]:
incorrect.head()

,original_text
0,"U wan me to ""chop"" seat 4 u nt?"
1,Yup. U reaching. We order some durian pastry a...
2,They become more ex oredi... Mine is like 25.....
3,I'm thai. what do u do?
4,Hi! How did your week go? Haven heard from you...


In [ ]:
correct.head()

,corrected_text
0,Do you want me to reserve seat for you or not?
1,Yeap. You reaching? We ordered some Durian pas...
2,They become more expensive already. Mine is li...
3,I'm Thai. What do you do?
4,Hi! How did your week go? Haven't heard from y...


In [ ]:
data_1 = pd.concat((incorrect, correct), axis=1)

In [ ]:
data_1

,original_text,corrected_text
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...
...,...,...
1995,Hmmm... Thk i usually book on wkends... Depend...,Hmm. I think I usually book on weekends. It de...
1996,ask them got any sms messages to gif me lei......,Can you ask them whether they have for any sms...
1997,We r near coca oredi...,We are near Coca already.
1998,hall Eleven. Got lectures le mah.èn forget abt...,Hall eleven. Got lectures. And forget about co...


In [ ]:
n_rows =  200
df1 = pd.read_csv('/content/drive/MyDrive/formal_text/C4_200M.tsv-00000-of-00010', names =['original_text','corrected_text'], sep = '\t', nrows = n_rows)
df2 = pd.read_csv('/content/drive/MyDrive/formal_text/C4_200M.tsv-00001-of-00010', names =['original_text','corrected_text'], sep = '\t', nrows = n_rows)
df3 = pd.read_csv('/content/drive/MyDrive/formal_text/C4_200M.tsv-00002-of-00010', names =['original_text','corrected_text'], sep = '\t', nrows = n_rows)
df4 = pd.read_csv('/content/drive/MyDrive/formal_text/C4_200M.tsv-00003-of-00010', names =['original_text','corrected_text'], sep = '\t', nrows = n_rows)
df5 = pd.read_csv('/content/drive/MyDrive/formal_text/C4_200M.tsv-00004-of-00010', names =['original_text','corrected_text'], sep = '\t', nrows = n_rows)
df6 = pd.read_csv('/content/drive/MyDrive/formal_text/C4_200M.tsv-00005-of-00010', names =['original_text','corrected_text'], sep = '\t', nrows = n_rows)
df7 = pd.read_csv('/content/drive/MyDrive/formal_text/C4_200M.tsv-00006-of-00010', names =['original_text','corrected_text'], sep = '\t', nrows = n_rows)
df8 = pd.read_csv('/content/drive/MyDrive/formal_text/C4_200M.tsv-00007-of-00010', names =['original_text','corrected_text'], sep = '\t', nrows = n_rows)
df9 = pd.read_csv('/content/drive/MyDrive/formal_text/C4_200M.tsv-00008-of-00010', names =['original_text','corrected_text'], sep = '\t', nrows = n_rows)
df10 = pd.read_csv('/content/drive/MyDrive/formal_text/C4_200M.tsv-00009-of-00010', names =['original_text','corrected_text'], sep = '\t', nrows = n_rows)
data_2 = pd.concat((df1,df2,df3,df4,df5,df6,df7,df8,df9,df10),axis = 0)
data = pd.concat((data_1, data_2), axis=0)

In [ ]:
data.shape

(4000, 2)

In [ ]:
data.head(-10)

,original_text,corrected_text
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...
...,...,...
185,The Mall's: Specially designed to accommodate ...,The Mall was “specially designed to accommodat...
186,"Inside of Historic City Hall, 830 Boonville Ave.","Inside Historic City Hall, 830 Boonville Ave."
187,McAfee Skyhigh Security Cloud for AWS: A cloud...,McAfee Skyhigh Security Cloud for AWS: A cloud...
188,This is not the first time the actors were ack...,This is not the first time the actors were ack...


In [ ]:
def preprocess_text_original(dataframe: pd.DataFrame, column: str):

  exclude = str.maketrans('', '', string.punctuation)

  dataframe[column] = dataframe[column].apply(lambda x: x.lower() if isinstance(x, str) else x)
  dataframe[column] = dataframe[column].apply(lambda x: x.replace(',',' COMMA '))
  dataframe[column] = dataframe[column].apply(lambda x: x.translate(exclude))
  #dataframe[column] = dataframe[column].apply(lambda x: x.split())


In [ ]:
def preprocess_text_corrected(dataframe: pd.DataFrame, column: str):

  exclude = str.maketrans('', '', string.punctuation)

  dataframe[column] = dataframe[column].apply(lambda x: x.lower() if isinstance(x, str) else x)
  dataframe[column] = dataframe[column].apply(lambda x: x.replace(',',' COMMA ') if isinstance(x, str) else x)
  dataframe[column] = dataframe[column].apply(lambda x: x.translate(exclude) if isinstance(x, str) else x)
  dataframe[column] = 'startseq ' + dataframe[column] + ' endseq'

  #dataframe[column] = dataframe[column].apply(lambda x : st_tok + ' ' + x + ' ' + end_tok)
  #dataframe[column] = dataframe[column].apply(lambda x: x.split())


In [ ]:
preprocess_text_original(data, 'original_text')
preprocess_text_corrected(data, 'corrected_text')

In [ ]:
data

,original_text,corrected_text
0,u wan me to chop seat 4 u nt,startseq do you want me to reserve seat for yo...
1,yup u reaching we order some durian pastry alr...,startseq yeap you reaching we ordered some dur...
2,they become more ex oredi mine is like 25 so h...,startseq they become more expensive already mi...
3,im thai what do u do,startseq im thai what do you do endseq
4,hi how did your week go haven heard from you f...,startseq hi how did your week go havent heard ...
...,...,...
195,this year brocon has lots of interesting geste...,startseq this year brocon has lots of interest...
196,in the school of cozza di legno i develop my c...,startseq in the school cozza di legno i improv...
197,usually COMMA im not a massive person ironing...,startseq usually COMMA im not a massive fan o...
198,you must also already entered the supplier in ...,startseq you must have also already entered th...


In [ ]:
data = data.sample(frac=1).reset_index(drop=True)

# DONT KNOW IF THIS MATTERS, CHECK IF SHUFFLE IS HAPPENING LATER #

In [ ]:
data

,original_text,corrected_text
0,shyla jagannantatha jansen pharmeticals COMMA ...,startseq shyla jagannatha COMMA janssen pharm...
1,welcome to resale therapy in palm beach garden...,startseq welcome to resale therapy in palm bea...
2,yes COMMA please feel free leave my story how...,startseq yes COMMA please feel free to use my...
3,hey can bring my dna notes tmr,startseq hey COMMA can you bring my dna notes...
4,yupz luv my tripweather is great too cant take...,startseq yup i love my trip weather is great t...
...,...,...
3995,hmm if its me i also wont feel like going lor ...,startseq hmm if it was me i would also feel li...
3996,harga centrifugal mill machines harga centrif...,startseq harga centrifugal mill machines harg...
3997,i dunno she say she wan pau,startseq i dont know she says that she wants b...
3998,all new zealanders are encouraged to occasiona...,startseq all new zealanders are encouraged to ...


In [ ]:
data['original_length'] = data['original_text'].apply(lambda x: len(x.split()) if isinstance(x, str) else x)
data['corrected_length'] = data['corrected_text'].apply(lambda x: len(x.split()) if isinstance(x, str) else x)

In [ ]:
#data = data[data['original_length'] >= 14]

In [ ]:
data

,original_text,corrected_text,original_length,corrected_length
0,shyla jagannantatha jansen pharmeticals COMMA ...,startseq shyla jagannatha COMMA janssen pharm...,6,9
1,welcome to resale therapy in palm beach garden...,startseq welcome to resale therapy in palm bea...,25,30
2,yes COMMA please feel free leave my story how...,startseq yes COMMA please feel free to use my...,13,16
3,hey can bring my dna notes tmr,startseq hey COMMA can you bring my dna notes...,7,11
4,yupz luv my tripweather is great too cant take...,startseq yup i love my trip weather is great t...,13,18
...,...,...,...,...
3995,hmm if its me i also wont feel like going lor ...,startseq hmm if it was me i would also feel li...,40,45
3996,harga centrifugal mill machines harga centrif...,startseq harga centrifugal mill machines harg...,67,49
3997,i dunno she say she wan pau,startseq i dont know she says that she wants b...,7,11
3998,all new zealanders are encouraged to occasiona...,startseq all new zealanders are encouraged to ...,30,33


In [ ]:
px.histogram(data, x='original_length', height=700, title='Original Sentences Length Distribution', marginal='box')

In [ ]:
px.histogram(data, x='corrected_length', height=700, title='Corrected Sentences Length Distribution', marginal='box')

In [ ]:
data = data[data['corrected_length'] <= 150]
data = data[data['original_length'] <= 150]

In [ ]:
data

,original_text,corrected_text,original_length,corrected_length
0,shyla jagannantatha jansen pharmeticals COMMA ...,startseq shyla jagannatha COMMA janssen pharm...,6,9
1,welcome to resale therapy in palm beach garden...,startseq welcome to resale therapy in palm bea...,25,30
2,yes COMMA please feel free leave my story how...,startseq yes COMMA please feel free to use my...,13,16
3,hey can bring my dna notes tmr,startseq hey COMMA can you bring my dna notes...,7,11
4,yupz luv my tripweather is great too cant take...,startseq yup i love my trip weather is great t...,13,18
...,...,...,...,...
3995,hmm if its me i also wont feel like going lor ...,startseq hmm if it was me i would also feel li...,40,45
3996,harga centrifugal mill machines harga centrif...,startseq harga centrifugal mill machines harg...,67,49
3997,i dunno she say she wan pau,startseq i dont know she says that she wants b...,7,11
3998,all new zealanders are encouraged to occasiona...,startseq all new zealanders are encouraged to ...,30,33


In [ ]:
max_length = 150

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

In [ ]:
X = data['original_text']
y = data['corrected_text']

In [ ]:
original_tokenizer = Tokenizer(oov_token='<OOV>')
original_tokenizer.fit_on_texts(X)
original_vocab_size = len(original_tokenizer.word_index) + 1

corrected_tokenizer = Tokenizer(oov_token='<OOV>')
corrected_tokenizer.fit_on_texts(y)
corrected_vocab_size = len(corrected_tokenizer.word_index) + 1

In [ ]:
corrected_vocab_size

12386

In [ ]:
original_vocab_size

14370

In [ ]:
### CHECK HOW TO USE TOKENIZER + DO YOU NEED TEST AND TRAIN??? ###
### CAN YOU PRE-PROCESS THEN SPLIT INTO TRAIN + TEST AFTER ###

In [ ]:
original = original_tokenizer.texts_to_sequences(X)
corrected = corrected_tokenizer.texts_to_sequences(y)
original = pad_sequences(original, maxlen=max_length, padding='post')
corrected = pad_sequences(corrected, maxlen=max_length, padding='post')

In [ ]:
BATCH_SIZE = 16

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(original, corrected, test_size=0.2, random_state=42)

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = train_data.shuffle(buffer_size=train_data.cardinality()).batch(BATCH_SIZE, drop_remainder=True)

test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_data = test_data.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
import tensorflow as tf
from keras.layers import Input, LSTM, Embedding, Dense, Bidirectional, Concatenate, Dot, Activation, TimeDistributed, GRU
from keras.models import Model

In [ ]:
# Show the mapping b/w word index and language tokenizer
def convert(lang, tensor):
  for t in tensor:
    if t != 0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

print ("Input Language; index to word mapping")
convert(original_tokenizer, X_train[1])
print ()
print ("Target Language; index to word mapping")
convert(corrected_tokenizer, y_train[1])

Input Language; index to word mapping
2590 ----> er
453 ----> nope
575 ----> sad
4 ----> to
138 ----> say
2 ----> comma
38 ----> im
238 ----> quite
7 ----> a
6071 ----> loner
23 ----> so
82 ----> just
6072 ----> pitched
7 ----> a
3571 ----> tent
4 ----> to
1193 ----> relax
150 ----> did
13 ----> you
57 ----> do
27 ----> this
166 ----> before

Target Language; index to word mapping
2 ----> startseq
1205 ----> nope
556 ----> sad
6 ----> to
176 ----> say
4 ----> comma
38 ----> im
215 ----> quite
11 ----> a
5628 ----> loner
26 ----> so
51 ----> just
5629 ----> pitched
11 ----> a
2511 ----> tent
6 ----> to
1315 ----> relax
113 ----> did
7 ----> you
42 ----> do
30 ----> this
143 ----> before
3 ----> endseq


In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
    super(Encoder, self).__init__()
    self.batch_size = batch_size
    self.enc_units = enc_units

    # Embed the vocab to a dense embedding
    self.embedding = Embedding(vocab_size, embedding_dim)

    # GRU Layer
    # glorot_uniform: Initializer for the recurrent_kernel weights matrix,
    # used for the linear transformation of the recurrent state
    self.gru = GRU(self.enc_units,
                   return_sequences=True,
                   return_state=True,
                   recurrent_initializer='glorot_uniform')

    self.LSTM(x, initial_state=hidden)

  # Encoder network comprises an Embedding layer followed by a GRU layer
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    return output, state

  # To initialize the hidden state
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_size, self.enc_units))

In [ ]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # values shape == (batch_size, max_len, hidden size)

    # we are doing this to broadcast addition along the time axis to calculate the score
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
# Decoder class
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_size):
    super(Decoder, self).__init__()
    self.batch_sz = batch_size
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # Used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # x shape == (batch_size, 1)
    # hidden shape == (batch_size, max_length)
    # enc_output shape == (batch_size, max_length, hidden_size)

    # context_vector shape == (batch_size, hidden_size)
    # attention_weights shape == (batch_size, max_length, 1)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state, attention_weights

In [ ]:
# Loss function
def loss_function(real, pred):

  # Take care of the padding. Not all sequences are of equal length.
  # If there's a '0' in the sequence, the loss is being nullified
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
### FIND OUT IF YOU CAN LOWER THE UNITS FOR HIDDEN LAYER ###
# SEE WHAT IT WAS IN ORIGINAL PAPER #
# THEN LOWER SO YOU CAN INCREASE SIZE OF DATASET #

In [ ]:
embedding_dim = 256
units = 1024

In [ ]:
encoder = Encoder(original_vocab_size, embedding_dim, units, BATCH_SIZE)

In [ ]:
decoder = Decoder(corrected_vocab_size, embedding_dim, units, BATCH_SIZE)

In [ ]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [ ]:
@tf.function
def train_step(original, corrected, enc_hidden):
  loss = 0

  # tf.GradientTape() -- record operations for automatic differentiation
  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(original, enc_hidden)

    # dec_hidden is used by attention, hence is the same enc_hidden
    dec_hidden = enc_hidden

    # <start> token is the initial decoder input

    #I believe that all my data has 'start' at beggining of mine

    dec_input = tf.expand_dims([corrected_tokenizer.word_index['<SOS>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, original.shape[1]):

      # Pass enc_output to the decoder
      predictions, dec_hidden, _, _ = decoder(dec_input, dec_hidden, enc_output)

      # Compute the loss
      loss += loss_function(corrected[:, t], predictions)

      # Use teacher forcing
      dec_input = tf.expand_dims(corrected[:, t], 1)

  # As this function is called per batch, compute the batch_loss
  batch_loss = (loss / corrected.shape[1])

  # Get the model's variables
  variables = encoder.trainable_variables + decoder.trainable_variables

  # Compute the gradients
  gradients = tape.gradient(loss, variables)

  # Update the variables of the model/network
  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

@tf.function
def test_step(original, corrected, enc_hidden):
    loss = 0
    enc_output, enc_hidden = encoder(original, enc_hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([corrected_tokenizer.word_index['<SOS>']] * BATCH_SIZE, 1)

    for i in range(1, corrected.shape[1]):
        predictions, dec_hidden, _, _ = decoder(dec_input, dec_hidden, enc_output)
        loss += loss_function(corrected[:, i], predictions)
        dec_input = tf.expand_dims(corrected[:, i], 1)

    batch_loss = (loss / corrected.shape[1])

    return batch_loss

In [ ]:
train_steps_per_epoch = len(X_train)//BATCH_SIZE
test_steps_per_epoch = len(X_test)//BATCH_SIZE

In [ ]:
len(train_data)

200

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
def train_model(train_data, test_data, epochs):

  start_time = timer()

  train_loss = []
  test_loss = []
  best_loss = np.inf

  # Training loop
  for epoch in range(epochs):

    # Initialize the hidden state
    enc_hidden = encoder.initialize_hidden_state()

    train_loss_num = 0.0
    test_loss_num = 0.0

    # Loop through the dataset
    for (batch, (original, corrected)) in enumerate(train_data.take(train_steps_per_epoch)):

      # Call the train method
      train_batch_loss = train_step(original, corrected, enc_hidden)

      # Compute the loss (per batch)
      train_loss_num += (train_batch_loss/train_steps_per_epoch)

    for (batch, (original, corrected)) in enumerate(test_data.take(test_steps_per_epoch)):
      # Call the test method
      test_batch_loss = test_step(original, corrected, enc_hidden)

      # Compute the loss (per batch)
      test_loss_num += (test_batch_loss/test_steps_per_epoch)

    if np.float32(test_loss_num) < best_loss:
      checkpoint.save(file_prefix = checkpoint_prefix)
      best_loss = np.float32(test_loss_num)

    train_loss.append(np.float32(train_loss_num))
    test_loss.append(np.float32(test_loss_num))

    print('EPOCH {} | train_loss: {} | test_loss: {}\n'.format(epoch+1,
                                                               train_loss_num,
                                                               test_loss_num))

  end_time = timer()

  print('Time taken for {} epochs: {} seconds'.format(epochs,
                                                      round((end_time - start_time),2)))

  return {'train_loss': train_loss, 'test_loss': test_loss}

In [ ]:
# ADD IN test_loss_num DIVIDED BY LEN OF TEST_DATALOADER + DO SAME FOR TRAN AND SEE IF THAT WORKS #

In [ ]:
results = train_model(train_data, test_data, epochs=10)

KeyboardInterrupt: ignored

In [ ]:
results = train_model(train_data, test_data, epochs=10)

EPOCH 1 | train_loss: 0.30347391963005066 | test_loss: 0.9826571941375732

EPOCH 2 | train_loss: 0.24908116459846497 | test_loss: 1.0176172256469727

EPOCH 3 | train_loss: 0.20422552525997162 | test_loss: 1.0348773002624512

EPOCH 4 | train_loss: 0.16728803515434265 | test_loss: 1.0493438243865967

EPOCH 5 | train_loss: 0.13709178566932678 | test_loss: 1.066292643547058

EPOCH 6 | train_loss: 0.112442746758461 | test_loss: 1.080802321434021

EPOCH 7 | train_loss: 0.09046052396297455 | test_loss: 1.0995951890945435

EPOCH 8 | train_loss: 0.07262834161520004 | test_loss: 1.1096075773239136

EPOCH 9 | train_loss: 0.05872933566570282 | test_loss: 1.127455234527588

EPOCH 10 | train_loss: 0.04734436422586441 | test_loss: 1.1474707126617432

Time taken for 10 epochs: 2029.81 seconds


In [ ]:
results

{'train_loss': [275.22525, 262.4667, 244.89789, 219.82285, 204.81032],
 'test_loss': [72.16209, 70.87148, 66.25543, 61.618362, 59.594826]}

In [ ]:
encoder = Encoder(original_vocab_size, embedding_dim, units, 64)
decoder = Decoder(corrected_vocab_size, embedding_dim, units, 64)
optimizer = tf.keras.optimizers.Adam()

In [ ]:
# loading
with open('tokenizer.pickle', 'rb') as handle:
    corrected_tokenizer = pickle.load(handle)

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/formal_text/training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [ ]:
checkpoint.restore(tf.train.latest_checkpoint('/content/training_checkpoints'))

In [ ]:
def preprocess_text(input: str):

  exclude = str.maketrans('', '', string.punctuation)

  input = input.lower()
  input = input.replace(',',' COMMA')
  input = input.translate(exclude)

  return input

In [ ]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(corrected_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
preprocess_text('at, work today!')

'at COMMA work today'

In [ ]:
'aT woRK TODay'.lower()

'at work today'

In [ ]:
def predict_sentence(input: str):

    input = preprocess_text(input)
    input = original_tokenizer.texts_to_sequences([input])
    input = pad_sequences(input, maxlen=max_length, padding='post')

    enc_hidden = [tf.zeros((1, units))]
    # CHECK WHEN YOU NEED TO CALL ENC_HIDDEN #

    enc_output, enc_hidden = encoder(input, enc_hidden)
    dec_hidden = enc_hidden

    current_token = tf.expand_dims([corrected_tokenizer.word_index['startseq']], 0)

    pred_sentence = ''

    for i in range(max_length):
        prediction, dec_hidden, _ = decoder(current_token, dec_hidden, enc_output)
        prediction = np.argmax(prediction[0])

        next_word = corrected_tokenizer.index_word[prediction]
        if next_word == 'endseq':
            break
        else:
            pred_sentence += ' ' + next_word
            current_token = tf.expand_dims([prediction], 0)

    return pred_sentence.strip()

In [ ]:
predict_sentence('thank you for reading. If you want to read some of more work of mine then please read my website')

'you must bring your student joss sticks do you are the right or what you want to trade the number and needles what you want the right or what you want to trade the number and needles what you want the right or what you want to trade the number and needles what you want the right or what you want to trade the number and needles what you want the right or what you want to trade the number and needles what you want the right or what you want to trade the number and needles what you want the right or what you want to trade the number and needles what you want the right or what you want to trade the number and needles what you want the right or what you want to trade the number and needles what you want the right or what you'

In [ ]:
for (batch, (original, corrected)) in enumerate(test_data.take(steps_per_epoch)):
    #print(original)
    train_batch_loss = test_step(original, corrected, enc_hidden)
    print(train_batch_loss)

tf.Tensor(1.1679786, shape=(), dtype=float32)
tf.Tensor(1.0654582, shape=(), dtype=float32)
tf.Tensor(0.8854652, shape=(), dtype=float32)
tf.Tensor(1.1904211, shape=(), dtype=float32)
tf.Tensor(0.96917707, shape=(), dtype=float32)
tf.Tensor(0.9806248, shape=(), dtype=float32)
tf.Tensor(0.95734835, shape=(), dtype=float32)
tf.Tensor(1.0250137, shape=(), dtype=float32)
tf.Tensor(1.0862771, shape=(), dtype=float32)
tf.Tensor(0.9626187, shape=(), dtype=float32)
tf.Tensor(1.1480637, shape=(), dtype=float32)
tf.Tensor(1.097323, shape=(), dtype=float32)
tf.Tensor(0.96657646, shape=(), dtype=float32)
tf.Tensor(1.1956592, shape=(), dtype=float32)
tf.Tensor(0.9822043, shape=(), dtype=float32)
tf.Tensor(1.0803487, shape=(), dtype=float32)
tf.Tensor(1.1185752, shape=(), dtype=float32)
tf.Tensor(1.1713077, shape=(), dtype=float32)
tf.Tensor(1.3180382, shape=(), dtype=float32)
tf.Tensor(0.99514395, shape=(), dtype=float32)
tf.Tensor(1.1025077, shape=(), dtype=float32)
tf.Tensor(1.2778717, shape=(), 

In [ ]:
input_tokens = []
for sentance in data['original_text']:
  for word in sentance:
    if word not in input_tokens:
      input_tokens.append(word)

target_tokens = []
for sentance in data['corrected_text']:
  for word in sentance:
    if word not in target_tokens:
      target_tokens.append(word)

In [ ]:
original_max_seq = np.max([len(i) for i in data['original_text']])
corrected_max_seq = np.max([len(i) for i in data['corrected_text']])

In [ ]:
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)
print(original_max_seq)
print(corrected_max_seq)

493
238


In [ ]:
pad_tok = 'PAD'
sep_tok = ' '
special_tokens = [pad_tok, sep_tok, st_tok, end_tok]
num_encoder_tokens += len(special_tokens)
num_decoder_tokens += len(special_tokens)

In [ ]:
num_encoder_tokens
num_decoder_tokens

84928

In [ ]:
input_token_index = {}
target_token_index = {}

for i,tok in enumerate(special_tokens):
  input_token_index[tok] = i
  target_token_index[tok] = i

In [ ]:
offset = len(special_tokens)
for i, tok in enumerate(input_tokens):
  input_token_index[tok] = i + offset
for i, tok in enumerate(target_tokens):
  target_token_index[tok] = i + offset

In [ ]:
reverse_input_tok_index = dict(
  (i, tok) for tok, i in input_token_index.items())
reverse_target_tok_index = dict(
  (i, tok) for tok, i in target_token_index.items())

In [ ]:
max_encoder_seq_length = 500
max_decoder_seq_length = 250

In [ ]:
encoder_input_data = np.zeros(
    (len(data['original_text']), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(data['corrected_text']), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(data['corrected_text']), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(data['original_text'], data['corrected_text'])):

  for t, tok in enumerate(input_text):
    encoder_input_data[i, t] = input_token_index[tok]
  encoder_input_data[i, t+1:] = input_token_index[pad_tok]

  for t, tok in enumerate(target_text):
          # decoder_target_data is ahead of decoder_input_data by one timestep

    decoder_input_data[i, t] = target_token_index[tok]
    if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[tok]] = 1.

    decoder_input_data[i, t+1:] = target_token_index[pad_tok]
    decoder_target_data[i, t:, target_token_index[pad_tok]] = 1.


In [ ]:
emb_sz = 100

In [ ]:
#from keras.utils import plot_model
#plot_model(model, show_shapes=True, show_layer_names=True)

In [ ]:
model, encoder_model, decoder_model = seq2seq_attention(num_encoder_tokens, num_decoder_tokens, emb_sz=emb_sz, latent_dim=emb_sz)
print(model.summary())

[<KerasTensor: shape=(None, 200) dtype=float32 (created by layer 'concatenate')>, <KerasTensor: shape=(None, 200) dtype=float32 (created by layer 'concatenate_1')>]


KerasTensor(type_spec=TensorSpec(shape=(None, None, 200), dtype=tf.float32, name=None), name='lstm_1/PartitionedCall:1', description="created by layer 'lstm_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, None, 200), dtype=tf.float32, name=None), name='bidirectional/concat:0', description="created by layer 'bidirectional'")
attention KerasTensor(type_spec=TensorSpec(shape=(None, None, None), dtype=tf.float32, name=None), name='attention/Softmax:0', description="created by layer 'attention'")
encoder-decoder  model:
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 100)            1029010

In [ ]:
data.shape

(52000, 2)

In [ ]:
encoder_input_data = dask.array.from_array(encoder_input_data, chunks=(52000,500))
decoder_input_data = dask.array.from_array(decoder_input_data, chunks=(52000,250))
decoder_target_data = dask.array.from_array(decoder_target_data, chunks=(52000, 250, 84928))

In [ ]:
encoder_input_data.shape

(52000, 500)

In [ ]:
decoder_input_data.shape

(52000, 250)

In [ ]:
decoder_target_data.shape

(52000, 250, 84928)

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=2,
          epochs=5,
)

Epoch 1/5
 1392/26000 [>.............................] - ETA: 1:31:43 - loss: 108.2991 - acc: 0.9981

KeyboardInterrupt: ignored

In [ ]:
model.predict(['hello, you idiot'])

ValueError: ignored